---

In [1]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd


/upb/users/z/zahera/profiles/unix/cs/.conda/envs/COTs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import openai

openai.api_key = ""

## The Student Model (falcon-7b quanitized)

In [3]:
#configuration of quantization of Falcon
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,

)


model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id, 
        device_map="auto",
        quantization_config=quantization_config,
        )

tokenizer = AutoTokenizer.from_pretrained(model_id)


# student pipeline for answering the questions
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=600,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

Loading checkpoint shards: 100%|██████████| 15/15 [00:04<00:00,  3.32it/s]


---
## Evaluation:


### Evaluation Prompt:

In [21]:
prompt_evaluation="""
- Act as an explainability evaluation metric and score the following answer based on the input question on a scale of 0.0-10.0
- Scoring criteria: The score of explainability represents the degree of precision in conveying the salient information to answer and explain the answer of the input question.
\n
- 0:  means that the text contains answer that is not relevant to the input_question. 
- 3: means the text is too general or not specific enough to answer the question
- 5: means that the text contains answer to the questions but it is not explainable
- 10: means that the text contains clear answer with facts and essential information to validate it's correctness as an explainable answer to the input_question. 

The score is 
"""
    
def evaluate_explainability(question, answer): 
    
    evaluation_message=[{"role": "user", "content": prompt_evaluation+"\nQuestion: "+question+"\n Answer=\n"+answer}]
    response = openai.ChatCompletion.create(
    model="gpt-4",
    messages = evaluation_message,
    temperature=0.0,
    max_tokens=300,
    frequency_penalty=0.0)

    score= response["choices"][0]['message']['content']
    
    return score    
    

## Prompt Tuning

In [6]:
prompt_teacher= """
- Act as a prompt engineer and change the text to be an informative Prompt.
- The goal is to improve the explainability_score to be equal or more than 9.0 by writing an informative prompt with Few_examples (e.g., one or two). 
- The examples are a set of questions that are different, but related to the input_question (but not the same) with their explainable answers.
- The maximum length of your should not exceed 300 tokens.

- The format of your response should be: 
< Examples: a list of pairs (question, it's explainable answer)>
\n
< Prompt : write here the changed prompt>
\n
< Input_question: write here the Input_question>
"""

def prompt_tuning(prompt_student,explainability_score) : 
    
    message=[{"role": "user", "content": prompt_teacher+"\n Text=\n"+prompt_student}]
    response = openai.ChatCompletion.create(
    model="gpt-4",
    messages = message,
    temperature=0.0,
    max_tokens=300,
    frequency_penalty=0.0
    )
    
    updated_prompt= response["choices"][0]['message']['content']
    
    return updated_prompt


### Teacher-Student Loop:

In [10]:
question = "Why do some people develop allergies to certain foods, while others do not?"

In [22]:
prompt_student="Answer the question: \nQ: "+question


def student_answer(prompt_student): 
    answer=pipeline(prompt_student)
    student_answer= answer[0]['generated_text'].replace(prompt_student, "")
    
    return student_answer

answer= student_answer(prompt_student)

score=evaluate_explainability(question,answer)
#score before prompt tuning
score

'8.0'

In [23]:
answer

'\nA: Allergies to certain foods are caused by an overreaction of the immune system. Some people may be born with a genetic predisposition to food allergies, while others may develop the condition in later life due to environmental factors.'

In [24]:
# loop for a fixed number of itera
max_iterations = 2
updated_prompt= prompt_student

while (float(score) <9.5 and max_iterations > 0): 
        
    updated_prompt=prompt_tuning(updated_prompt,score)
    
    # Ask the student
    answer= student_answer(updated_prompt)
    
    #compute the new score of explainability
    score=evaluate_explainability(question,answer)
    
    max_iterations-=1

In [25]:
score

'10.0'

In [27]:
print (updated_prompt)

Examples: 
1. Question: Why are some people lactose intolerant while others are not?
   Answer: Lactose intolerance is a common digestive disorder where the body is unable to fully digest lactose, a sugar found in milk and dairy products. This is caused by a deficiency in lactase, an enzyme produced by the small intestine that breaks down lactose. The amount of lactase produced can vary greatly between individuals due to genetic factors, which explains why some people are lactose intolerant while others are not.

2. Question: Why do some people have gluten intolerance while others don't?
   Answer: Gluten intolerance, or celiac disease, is a condition where the body's immune system reacts to gluten, a protein found in wheat, barley, and rye. This reaction damages the lining of the small intestine, leading to various digestive problems. The exact cause of celiac disease is unknown, but it is believed to involve a combination of genetic and environmental factors. This is why some people 

In [28]:
answer

" There are a few factors that can contribute to the development of food allergies, including genetics, immune responses, and environmental influences. \n\nA primary reason why one person may develop a food allergy while another does not may have to do with a genetic component. Inherited traits can determine whether a person develops a heightened immune response to certain substances. This can result in allergic reactions, including the development of food allergies. \n\nEnvironmental factors that can affect the development of food allergies include exposure to allergens during infancy, such as breastfed infants who develop an allergy to breast milk proteins. Additionally, exposure to foods at an early age can increase a child's likelihood of developing food allergies later in life. \n\nEnvironmental factors that may contribute to the development of food allergies include a lack of exposure to allergens or a low level of immunological tolerance to particular substances. This is why som